In [1]:
%matplotlib inline

import os
from cycler import cycler
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import plotly.plotly as py
import plotly.tools as tls
from plotly.graph_objs import *
from parsing.argus_analysis import *


tls.set_credentials_file(username='cguldner', api_key=os.environ['PLOTLY_API_KEY'])

In [2]:
daemon_path = 'data/argusout/'
module_path = 'data/moduleout/'

In [3]:
basic_plot_params = {
   'axes.labelsize': 14,
   'font.size': 15,
   'legend.fontsize': 11,
   'legend.handlelength': 2,
   'xtick.labelsize': 12,
   'ytick.labelsize': 12,
   'lines.marker': None,
   'text.usetex': False,
   'figure.figsize': [20, 20],
   'axes.prop_cycle': cycler('color', ['b', 'g', 'r', 'c', 'm', 'y', 'k']) +
                       cycler('linestyle', ['-', '--', ':', '-.', '-', '--', ':'])
}

plt.rcParams.update(basic_plot_params)

In [4]:
def hide_nth_xtick(ax, n):
    [l.set_visible(False) for (i,l) in enumerate(ax.xaxis.get_ticklabels()) if i % n != 0]

In [27]:
def plot_daemon_stats(stats, node_names):
    cpu = plt.subplot2grid((3,3), (0,0), colspan=3)
    cpu.set_ylabel('CPU usage')
    cpu.set_title('Percentage of CPU usage')
    
    mem = plt.subplot2grid((3,3), (1,0), colspan=3)
    mem.set_ylabel('Memory (in bytes)')
    mem.set_title('Active Memory usage')

    failed = plt.subplot2grid((3,3), (2, 0))
    failed.set_ylabel('Puzzles')
    failed.set_title('Number of Puzzles Failed')

    received = plt.subplot2grid((3,3), (2, 1))
    received.set_ylabel('Puzzles')
    received.set_title('Number of Puzzles Received')

    sent = plt.subplot2grid((3,3), (2, 2))
    sent.set_ylabel('Puzzles')
    sent.set_title('Number of Puzzles Sent')
    
    for node in node_names:
        cur_node = stats.get(node)
        try:
            cpu.plot(cur_node['Timestamp'], cur_node['cpu_percent'], label=node)
            mem.plot(cur_node['Timestamp'], cur_node['active_memory'], label=node)
            failed.plot(cur_node['Timestamp'], cur_node['ChallengeFailed'], label=node)
            received.plot(cur_node['Timestamp'], cur_node['ChallengeRecvd'], label=node)
            sent.plot(cur_node['Timestamp'], cur_node['ChallengeSent'], label=node)
        except KeyError:
            continue

    for plot in [cpu, mem, failed, received, sent]:
        plot.set_xlabel('Time')
        plot.yaxis.grid()
        
    fig = plt.gcf()
    fig.tight_layout(pad=1, h_pad=3.5)
    
    plotly_fig = tls.mpl_to_plotly(fig)
    update = dict(
        layout=dict(showlegend=True)
    )
    plot_url = py.plot_mpl(fig, update=update, filename='daemon_data')

In [28]:
daemon_stats = {}
files = [os.path.splitext(file) for file in os.listdir(daemon_path)]
for file in files:
    daemon_stats[file[0]], sample_rate = get_daemon_stats_from_file(daemon_path + ''.join(file))

plot_daemon_stats(daemon_stats, [f[0] for f in files])

The draw time for this plot will be slow for clients without much RAM.


In [53]:
def plot_module_stats(stats, file_names):
    fig = plt.figure()

    listen_q = fig.add_subplot(211)
    accept_q = fig.add_subplot(212)
    for out_file in stats:
        queue = stats.get(out_file)
        listen_q.set_ylabel('Queue Size (in ?????)')
        listen_q.set_title('Listen Queue Size')

        accept_q.set_ylabel('Queue Size (in ?????)')
        accept_q.set_title('Accept Queue Size')

        listen_q.plot(queue['Timestamp'], queue['listen_q'], label=out_file)
        accept_q.plot(queue['Timestamp'], queue['accept_q'], label=out_file)

    for plot in [listen_q, accept_q]:
        plot.set_xlabel('Time')
        plot.yaxis.grid()
    
    fig.tight_layout(pad=1, h_pad=3.5)

    plotly_fig = tls.mpl_to_plotly(fig)
    update = dict(
        layout=dict(showlegend=True)
    )
#     for plot in plotly_fig['data']:
#         plot['legendgroup'] = ''
#         plot['legendgroup'].update(plot['name'])
#         print plot
#         plotly_fig['layout']['titlefont'].update({'color':'blue', 'size':20, 'family':'monospace'})

    plot_url = py.iplot_mpl(fig, update=update, filename='module_data')

In [54]:
module_stats = {}
files = [os.path.splitext(file) for file in os.listdir(module_path)]
for file in files:
    module_stats[file[0]] = get_module_stats_from_file(module_path + ''.join(file))
    
plot_module_stats(module_stats, [f[0] for f in files])